In [1]:
import sys
sys.path.append('../')

In [2]:
from utils.utils import Utils
from utils.preprocess import Preprocess

In [3]:
%%time
utils = Utils()
pre = Preprocess(mode="normalize")

CPU times: user 20.2 s, sys: 2.41 s, total: 22.6 s
Wall time: 37.4 s


In [12]:
%%time
data = utils.read_data("dataset/track-1-essay-empathy-train.csv")
data

CPU times: user 22 ms, sys: 0 ns, total: 22 ms
Wall time: 21.4 ms


,message_id,response_id,empathy,distress,empathy_bin,distress_bin,emotion_label,essay,gender,education,...,income,personality_conscientiousness,personality_openess,personality_extraversion,personality_agreeableness,personality_stability,iri_perspective_taking,iri_personal_distress,iri_fantasy,iri_empathatic_concern
0,R_1hGrPtWM4SumG0U_1,R_1hGrPtWM4SumG0U,5.667,4.375,1,1,sadness,it is really diheartening to read about these ...,1,4,...,50000,6.0,5.0,5.0,5.5,5.5,3.571,2.000,3.429,4.000
1,R_1hGrPtWM4SumG0U_2,R_1hGrPtWM4SumG0U,4.833,4.875,1,1,joy,the phone lines from the suicide prevention li...,1,4,...,50000,6.0,5.0,5.0,5.5,5.5,3.571,2.000,3.429,4.000
2,R_1hGrPtWM4SumG0U_3,R_1hGrPtWM4SumG0U,5.333,3.500,1,0,neutral,"no matter what your heritage, you should be ab...",1,4,...,50000,6.0,5.0,5.0,5.5,5.5,3.571,2.000,3.429,4.000
3,R_1hGrPtWM4SumG0U_4,R_1hGrPtWM4SumG0U,4.167,5.250,1,1,fear,it is frightening to learn about all these sha...,1,4,...,50000,6.0,5.0,5.0,5.5,5.5,3.571,2.000,3.429,4.000
4,R_1hGrPtWM4SumG0U_5,R_1hGrPtWM4SumG0U,5.333,4.625,1,1,neutral,the eldest generation of russians aren't being...,1,4,...,50000,6.0,5.0,5.0,5.5,5.5,3.571,2.000,3.429,4.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1855,R_DHy2Rcz9Hym8jgl_1,R_DHy2Rcz9Hym8jgl,1.000,1.000,0,0,joy,"These days, women are winning in sports. There...",2,4,...,30000,7.0,7.0,7.0,7.0,7.0,5.000,1.286,4.286,4.429
1856,R_DHy2Rcz9Hym8jgl_2,R_DHy2Rcz9Hym8jgl,1.000,1.000,0,0,anger,I hate ISIS. They are a group full of hate and...,2,4,...,30000,7.0,7.0,7.0,7.0,7.0,5.000,1.286,4.286,4.429
1857,R_DHy2Rcz9Hym8jgl_3,R_DHy2Rcz9Hym8jgl,1.000,1.000,0,0,disgust,This is just disgusting. I cannot believe that...,2,4,...,30000,7.0,7.0,7.0,7.0,7.0,5.000,1.286,4.286,4.429
1858,R_DHy2Rcz9Hym8jgl_4,R_DHy2Rcz9Hym8jgl,1.000,1.000,0,0,surprise,I feel like the world is so corrupt that it no...,2,4,...,30000,7.0,7.0,7.0,7.0,7.0,5.000,1.286,4.286,4.429


In [5]:
post_no = 1141
data.essay.values.tolist()[post_no]

"I totally get this in a lot of ways. I used to roll my eyes at people who were frothing at the mouth over Obama being elected, and how they seemed to make grand statements of the world ending RIGHT NOW because a black man was in the white house. Now...I don't know. I see my friends and family freaking out and I want to tell them to just chill out. Obama didn't destroy America--the right wingers were wrong about that. Neither will Trump. Life will keep going. It brings a lot of stress listening to people freaking out all the time. And yet, I get it. I get why they are worried and freaking out. I don't like it either, but I don't get the point of stressing out so much. But then again, I've completely dissociated from all this. I don't really read the news anymore."

In [6]:
%%time
normalized_data = pre.normalize_corpus(data.copy(), column_name="essay")
normalized_data

CPU times: user 20.7 s, sys: 246 ms, total: 20.9 s
Wall time: 32.9 s


,message_id,response_id,empathy,distress,empathy_bin,distress_bin,emotion_label,essay,gender,education,...,income,personality_conscientiousness,personality_openess,personality_extraversion,personality_agreeableness,personality_stability,iri_perspective_taking,iri_personal_distress,iri_fantasy,iri_empathatic_concern
0,R_1hGrPtWM4SumG0U_1,R_1hGrPtWM4SumG0U,5.667,4.375,1,1,sadness,it is really disheartening to read about thes...,1,4,...,50000,6.0,5.0,5.0,5.5,5.5,3.571,2.000,3.429,4.000
1,R_1hGrPtWM4SumG0U_2,R_1hGrPtWM4SumG0U,4.833,4.875,1,1,joy,the phone lines from the suicide prevention l...,1,4,...,50000,6.0,5.0,5.0,5.5,5.5,3.571,2.000,3.429,4.000
2,R_1hGrPtWM4SumG0U_3,R_1hGrPtWM4SumG0U,5.333,3.500,1,0,neutral,"no matter what your heritage , you should be ...",1,4,...,50000,6.0,5.0,5.0,5.5,5.5,3.571,2.000,3.429,4.000
3,R_1hGrPtWM4SumG0U_4,R_1hGrPtWM4SumG0U,4.167,5.250,1,1,fear,it is frightening to learn about all these sh...,1,4,...,50000,6.0,5.0,5.0,5.5,5.5,3.571,2.000,3.429,4.000
4,R_1hGrPtWM4SumG0U_5,R_1hGrPtWM4SumG0U,5.333,4.625,1,1,neutral,the eldest generation of russians are not bei...,1,4,...,50000,6.0,5.0,5.0,5.5,5.5,3.571,2.000,3.429,4.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1855,R_DHy2Rcz9Hym8jgl_1,R_DHy2Rcz9Hym8jgl,1.000,1.000,0,0,joy,"These days , women are winning in sports . Th...",2,4,...,30000,7.0,7.0,7.0,7.0,7.0,5.000,1.286,4.286,4.429
1856,R_DHy2Rcz9Hym8jgl_2,R_DHy2Rcz9Hym8jgl,1.000,1.000,0,0,anger,I hate ISIS . They are a group full of hate a...,2,4,...,30000,7.0,7.0,7.0,7.0,7.0,5.000,1.286,4.286,4.429
1857,R_DHy2Rcz9Hym8jgl_3,R_DHy2Rcz9Hym8jgl,1.000,1.000,0,0,disgust,This is just disgusting . I cannot believe th...,2,4,...,30000,7.0,7.0,7.0,7.0,7.0,5.000,1.286,4.286,4.429
1858,R_DHy2Rcz9Hym8jgl_4,R_DHy2Rcz9Hym8jgl,1.000,1.000,0,0,surprise,I feel like the world is so corrupt that it n...,2,4,...,30000,7.0,7.0,7.0,7.0,7.0,5.000,1.286,4.286,4.429


In [7]:
normalized_data.essay.values.tolist()[post_no]

' I totally get this in a lot of ways . I used to roll my eyes at people who were frothing at the mouth over Obama being elected , and how they seemed to make grand statements of the world ending RIGHT NOW because a black man was in the white house . Now ... I do not know . I see my friends and family freaking out and I want to tell them to just chill out . Obama did not destroy America--the right wingers were wrong about that . Neither will Trump . Life will keep going . It brings a lot of stress listening to people freaking out all the time . And yet , I get it . I get why they are worried and freaking out . I do not like it either , but I do not get the point of stressing out so much . But then again , I have completely dissociated from all this . I do not really read the news anymore .'

In [10]:
normalized_data.to_csv("dataset/track-1-essay-empathy-train-normalized.csv", index=False)

In [ ]:
res = pre.clean_text(data.essay.values.tolist()[post_no], 
                     remove_stopwords=True,
                     lemmatize=True)
print(res)

In [ ]:
res = pre.clean_text(data.essay.values.tolist()[post_no], 
                     remove_stopwords=True,
                     lemmatize=False)
print(res)

In [ ]:
res = pre.clean_text(data.essay.values.tolist()[post_no], 
                     remove_stopwords=False,
                     lemmatize=True)
print(res)

In [ ]:
res = pre.clean_text(data.essay.values.tolist()[post_no], 
                     remove_stopwords=False,
                     lemmatize=False)
print(res)